In [20]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader # for the dataloader
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report

In [1]:
# Convert dataframe into dictionary of text and labels
def reader(df):
    texts = df['text'].values.tolist()
    labels = df['label'].values.tolist()

    return {'texts':texts, 'labels':labels}

In [ ]:
df = pd.read_csv('data/emotionlabeled_train.csv')

In [27]:
# DataLoader
class OlidDataset(Dataset):
  def __init__(self, tokenizer, input_set):
    # input_set: dictionary version of the df
    self.texts = input_set['texts']
    self.labels = input_set['labels']
    self.tokenizer = tokenizer

  def collate_fn(self, batch):
    texts = []
    labels = []

    for b in batch:
      texts.append(str(b['text']))
      labels.append(b['label'])

    print(texts)
    print(labels)
    encodings = self.tokenizer(
      texts,                        # what to encode
      return_tensors = 'pt',        # return pytorch tensors
      add_special_tokens = True,    # incld tokens like [SEP], [CLS]
      padding = "max_length",       # pad to max sentence length
      truncation = True,            # truncate if too long
      max_length= 128)              

    encodings['labels'] = torch.tensor(labels)
    return encodings

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    item = {'text': self.texts[idx], 'label': self.labels[idx]}

    return item


In [34]:
# train model
def train_model():
    optimizer = 'AdamW'
    learning_rate = 1.35e-04
    epochs = 1
    
    model_args = ClassificationArgs(num_train_epochs=epochs, 
                                          no_save=True, 
                                          no_cache=True, 
                                          overwrite_output_dir=True,
                                          learning_rate=learning_rate,
                                          optimizer=optimizer)

    model = ClassificationModel(model_type="roberta", 
                              model_name="roberta-base", 
                              args = model_args, 
                              num_labels=4, 
                              use_cuda=cuda_available)
    
    print(model)
    model.train_model(df_train[['text', 'labels']])

    print('model train finished')
    # Validation Set (Internal)
    y_pred, _ = model.predict(df_val.text.tolist())
    y_true = df_val['labels']
    print("Classification metrics for validation set")
    print(classification_report(y_true, y_pred))
    
    # Test Set (Internal)
    y_pred, _ = model.predict(df_test.text.tolist())
    y_true = df_test['labels']
    print("Classification metrics for test set")
    print(classification_report(y_true, y_pred))
    
    # # Test Set (External)
    # y_pred, _ = model.predict(df_submission.text.tolist()) 
    # labels2file([[k] for k in y_pred], 'task1.txt')

In [42]:
GPU = True

if GPU:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(f"Using {device}")

cuda_available = torch.cuda.is_available()
df_train = pd.read_csv('data/emotionlabeled_train.csv', index_col=0).head(10)
df_val = pd.read_csv('data/emotionlabeled_val.csv', index_col=0).head(5)
df_test = pd.read_csv('data/emotionlabeled_test.csv', index_col=0).head(5)

# print(df_train[['text', 'labels']])
# col_names = df_test.columns

train_model()

Using cpu


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/10 [00:00<?, ?it/s]

/Users/alicialawjy/Desktop/multilingual-psychotherapy-chatbot/satbot/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

/Users/alicialawjy/Desktop/multilingual-psychotherapy-chatbot/satbot/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alicialawjy/Desktop/multilingual-psychotherapy-chatbot/satbot/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alicialawjy/Desktop/multilingual-psychotherapy-chatbot/satbot/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_di

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Classification metrics for test set
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      0.25      0.40         4
           3       0.00      0.00      0.00         0

    accuracy                           0.20         5
   macro avg       0.33      0.08      0.13         5
weighted avg       0.80      0.20      0.32         5



/Users/alicialawjy/Desktop/multilingual-psychotherapy-chatbot/satbot/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alicialawjy/Desktop/multilingual-psychotherapy-chatbot/satbot/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alicialawjy/Desktop/multilingual-psychotherapy-chatbot/satbot/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_di